In [1]:
import pandas as pd
import numpy as np
import random
import re
from tkinter import *

In [2]:
movies= pd.read_csv("movies.csv",usecols=['movieId','title'])
movies

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017)
9738,193583,No Game No Life: Zero (2017)
9739,193585,Flint (2017)
9740,193587,Bungo Stray Dogs: Dead Apple (2018)


In [3]:
ratings=pd.read_csv("ratings.csv",usecols=['userId','movieId','rating'])
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [4]:
def cosine(test,fd):
    test=test.to_numpy()
    tp=fd.to_numpy()
    k=len(tp[0])-1
    simi=[]
    for i in range(len(tp)):
        c=0
        for j in range(len(tp[0])-2):
            c+=tp[i][j]*test[j]
        cosine=c/float(tp[i][k]*test[k])
        simi.append(1-cosine)
    id=fd.index.tolist()
    simi=list(zip(id,simi))
    return simi

In [5]:
def KNN(mn,n,fd):
    cd=cosine(mn,fd)
    distance=list(cd)
    distance.sort(key=lambda x: x[1])
    nb = []
    for i in range(n):
        nb.append(distance[i+1][0])
    frame = []
    for i in nb:
        idx=movies[movies['movieId']==i].index
        frame.append(movies.iloc[idx]['title'])
    frame=np.array(frame)
    df=pd.DataFrame(frame,index=range(1,n+1),columns=['Title'])
    return df

In [6]:
def name(mn):
    t=mn.split()
    mn=''
    for i in range(len(t)):
        mn+=t[i].capitalize()+' '
    return mn

In [7]:
def getRecommendation(movie_name,movie_count):
    movie_name=name(movie_name)
    movie_list=movies[movies['title'].str.contains(movie_name)]
    if len(movie_list):
        fd=ratings.pivot(index='movieId',columns='userId',values='rating')
        fd.fillna(0,inplace=True)
        temp=np.square(fd)
        temp=temp.sum(axis=1)
        temp=np.sqrt(temp)
        temp=temp.to_numpy()
        fd['sum']=temp
        movie_idx=movie_list.iloc[0]['movieId']
        movie_recommended=KNN(fd.loc[movie_idx],movie_count,fd)
        return movie_recommended
    else:
        return 'No Movie Found !!!\n\nCheck Your Movie Name....'

In [8]:
def avg():
    mx=ratings.pivot(index='movieId',columns='userId',values='rating')
    n=mx.count()
    u_avg=mx.sum(axis=0)
    u_avg=u_avg.div(n)
    for i in range(1,len(u_avg)+1):
        mx.loc[:,i].fillna(u_avg[i],inplace=True)
    mx=round(mx,1)
    c=len(mx.columns)
    m_avg=mx.sum(axis=1)
    m_avg=m_avg.div(c)
    m_avg=round(m_avg,1)
    m_avg.sort_values(inplace=True,ascending=False)
    m_avg=m_avg.loc[m_avg[:]>3.7]
    midx=m_avg.index.tolist()
    res=[]
    for i in midx:
        mid=movies[movies['movieId']==i].index
        res.append(movies.iloc[mid]['title'])
    res=np.array(res)
    return res

In [9]:
def randomcolor():
    color='#'
    for i in range(6):
        color+=random.choice('0123456789ABCDEF')
    return color

In [11]:
root = Tk()
root.geometry("800x800")
root.title('Recommendation System')
root.resizable(False,False)
root.configure(bg=randomcolor())
root.iconbitmap('icon.ico')

frame=Frame(root).place(x=20,y=0,height=400,width=760)

l1= Label(frame,text="MOVIE RECOMMENDATION SYSTEM",font=("Impact",30,"bold","italic","underline"),fg=randomcolor())
l1.place(x=100,y=10)
l2= Label(frame,text="Enter Your Favourite Movie : ",font=("Impact",20,'underline'),fg=randomcolor())
l2.place(x=120,y=75)
l3= Label(frame,text="Enter No of Movies : ",font=("Impact",20,'underline'),fg=randomcolor())
l3.place(x=120,y=145)

e1=Entry(frame,fg=randomcolor(),borderwidth=5,width=40,font=("Tisa",15))
e1.place(x=450,y=80,height=40,width=200)
e2=Entry(frame,fg=randomcolor(),borderwidth=5,font=("Tisa",15))
e2.place(x=450,y=150, height=40,width=80)

t=Text(root,borderwidth=5,wrap=WORD,padx=40,pady=40,font=("Tisa",15),state='disabled')
t.place(x=40,y=290,height=505,width=720)

r = IntVar()

def radio():
    if(r.get()):
        e1.configure(state='disabled')
        e2.configure(state='disabled')
    else:
        e1.configure(state='normal')
        e2.configure(state='normal')
        
r1=Radiobutton(frame,text='UCF',variable=r,value=1,borderwidth=5,font=("Impact",20),fg=randomcolor(),activeforeground=randomcolor(),command=radio)
r1.place(x=150,y=215)
r2=Radiobutton(frame,text='ICF',variable=r,value=0,borderwidth=5,font=("Impact",20),fg=randomcolor(),activeforeground=randomcolor(),command=radio)
r2.place(x=550,y=215)

def myClick():
    if(r.get()):
        res=avg()
        k=len(res)
        c=random.randint(0, k)
        t.configure(state='normal',fg=randomcolor(),font=("Tisa",15),pady=60,padx=80)
        t.delete(1.0,END)
        for i in range(10):
            l=str(10-i)
            if(len(l)!=1):
                 t.insert(0.0,l+'.  '+res[(c+i)%k][0]+'\n') 
            else:
                t.insert(0.0,' '+l+'.  '+res[(c+i)%k][0]+'\n')
    else:
        if(re.match("^[a-zA-Z]+[a-zA-Z0-9]*|^[0-9]+[a-zA-Z]+|^0+[a-bA-Z]*",e2.get()) ):
            t.configure(fg='red',padx=200,pady=100,font=("Tisa",18,'bold'),state='normal')
            t.delete(1.0,END)
            t.insert(0.0,'Wrong Count value!!!\n\nFill Correct Value...\n')
        else:
            if(e1.get()=='' or e2.get()==''):
                if(e1.get()=='' and e2.get()==''):
                    t.configure(fg='red',padx=200,pady=100,font=("Tisa",18,'bold'),state='normal')
                    t.delete(1.0,END)
                    t.insert(0.0,'Empty Response !!!\n\nFill Above Details...\n')
                elif(e1.get()==''):
                    t.configure(fg='red',padx=200,pady=100,font=("Tisa",18,'bold'),state='normal')
                    t.delete(1.0,END)
                    t.insert(0.0,'Empty Movie Name !!!\n\nFill Movie Name...\n')
                else:
                    t.configure(fg='red',padx=200,pady=100,font=("Tisa",18,'bold'),state='normal')
                    t.delete(1.0,END)
                    t.insert(0.0,'Empty Movies Count !!!\n\nFill Movies Count...\n')
            else:
                t.configure(state='normal')
                df=getRecommendation(e1.get(),int(e2.get()))
                if(isinstance(df,str)):
                    t.configure(fg='red',padx=200,pady=100,font=("Tisa",15,'bold'))
                    t.delete(1.0,END)
                    t.insert(0.0,df)
                else:
                    j=n=int(e2.get())+1
                    if(j>17):
                        t.configure(font=("Tisa",13),pady=30,padx=100)
                    else:
                        t.configure(font=("Tisa",15),pady=60,padx=80)
                    t.configure(fg=randomcolor())
                    t.delete(1.0,END)
                    for i in range(j-1):
                        k=str(j-i-1)
                        if(len(k)!=1):
                            t.insert(0.0,k+'.  '+df.iloc[i]['Title']+'\n') 
                        else:
                            t.insert(0.0,' '+k+'.  '+df.iloc[i]['Title']+'\n')
        t.configure(state='disabled')

b=Button(frame,text="Search",command=myClick,bg=randomcolor(),activebackground=randomcolor(),fg='white',borderwidth=7,font=("Impact",15))
b.place(x=350,y=220,width=80,height=40)

root.mainloop()

KeyboardInterrupt: 

In [ ]:
 movie_list=movies[movies['title'].str.contains(name('god father'))]

In [11]:
fd=ratings.pivot(index='movieId',columns='userId',values='rating')
fd

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
